In [1]:
import ibis

In [2]:
db = ibis.duckdb.connect('mortgage.db')

In [3]:
db.list_tables()

/Users/hussainsultan/workspace/ibis/ibis/backends/base/sql/alchemy/__init__.py:95: SAWarning: Dialect postgresql:psycopg2 will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  tables = self._inspector.get_table_names(


['duckdb_types',
 'duckdb_schemas',
 'duckdb_constraints',
 'sqlite_temp_schema',
 'sqlite_temp_master',
 'sqlite_schema',
 'sqlite_master',
 'duckdb_indexes',
 'duckdb_tables',
 'pragma_database_list',
 'duckdb_views',
 'duckdb_columns',
 'perf',
 'acq']

In [4]:
perf = db.table('perf')
acq = db.table('acq')
acq = acq[acq.loan_id, acq.orig_date.split('/')[1].name("year"), acq.borrower_credit_score] 

In [5]:
joined = acq.inner_join(perf, acq.loan_id == perf.loan_id)

In [6]:
dq30 = (
    ibis.case()
    .when(perf.current_loan_delinquency_status > 1, 1)
    .else_(0)
    .end()
    .name("dq30"))

chargeoffs = (
    ibis.case()
    .when((perf.zero_balance_code.isin(['02','03','09', '15'])) & (perf.disposition_date.notnull()), 1)
    .else_(0)
    .end()
    .name("charegoffs"))

dollar_co = (perf.zero_balance_code.isin(['02','03','09', '15']) &
            (perf.disposition_date.notnull())).ifelse(perf.current_actual_upb, 0)

In [16]:
loans = (joined
             .mutate(chargeoffs=chargeoffs, dollar_co=dollar_co )
             .projection([perf.loan_id, 
                             # dq30, 
                             chargeoffs,
                             dollar_co.name("dollar_co"),
                             perf.loan_age, 
                             perf.current_actual_upb,
                             acq.year,
                             acq.borrower_credit_score,
                         ]))

summary = (loans[loans.loan_age>0]
           .groupby([loans.year, loans.loan_age])
           .aggregate(co_count=lambda x: x.charegoffs.cast('int64').sum(),
                      dollar_co = lambda x: x.dollar_co.sum(),
                      avg_credit_score = lambda x: x.borrower_credit_score.mean(),
                      upb_sum = lambda x: x.current_actual_upb.sum()))         

acq_agg = acq.groupby([acq.year]).loan_id.count()

summary = summary.inner_join(acq_agg, acq_agg.year == summary.year)
summary = summary.projection([summary.year_x, summary.loan_age, summary["count(loan_id)"], summary.avg_credit_score, summary.upb_sum, 
                             summary.dollar_co])

In [17]:
print(summary.compile().compile(compile_kwargs={"literal_binds": True}))

WITH t0 AS 
(SELECT t2.loan_id AS loan_id, list_element(string_to_array(t2.orig_date, '/'), 1) AS year, t2.borrower_credit_score AS borrower_credit_score 
FROM acq AS t2)
 SELECT t1.year_x, t1.loan_age, t1."count(loan_id)", t1.avg_credit_score, t1.upb_sum, t1.dollar_co 
FROM (SELECT t2.year AS year_x, t2.loan_age AS loan_age, t2.avg_credit_score AS avg_credit_score, t2.co_count AS co_count, t2.dollar_co AS dollar_co, t2.upb_sum AS upb_sum, t3.year AS year_y, t3."count(loan_id)" AS "count(loan_id)" 
FROM (SELECT t4.year AS year, t4.loan_age AS loan_age, avg(t4.borrower_credit_score) AS avg_credit_score, sum(CAST(t4.charegoffs AS BIGINT)) AS co_count, sum(t4.dollar_co) AS dollar_co, sum(t4.current_actual_upb) AS upb_sum 
FROM (SELECT t5.loan_id AS loan_id, CASE WHEN (t5.zero_balance_code IN ('02', '03', '09', '15') AND t5.disposition_date IS NOT NULL) THEN 1 ELSE 0 END AS charegoffs, CASE WHEN (t5.zero_balance_code IN ('02', '03', '09', '15') AND t5.disposition_date IS NOT NULL) THEN t5.

In [15]:
%%time
df = summary.execute()

CPU times: user 52min 3s, sys: 3min 56s, total: 56min
Wall time: 9min 8s


In [37]:
df

,year,loan_age,avg_credit_score,co_count,dollar_co,upb_sum
0,2003,67.0,735.462844,425,47239927.28,3.324908e+11
1,2015,15.0,754.320344,17,3276420.53,3.584938e+11
2,2007,9.0,721.543722,145,30825612.83,2.206895e+11
3,2003,1.0,729.628530,0,0.00,1.543691e+09
4,2015,16.0,754.371683,20,2895264.10,3.530540e+11
...,...,...,...,...,...,...
2251,2012,69.0,770.612153,0,0.00,3.065800e+10
2252,2012,66.0,769.964599,1,87658.00,8.499493e+10
2253,2012,70.0,769.715452,0,0.00,1.317821e+10
2254,2012,67.0,770.412095,0,0.00,6.678450e+10


In [15]:
%%time
summary.execute()

CPU times: user 2h 44min 23s, sys: 5min 18s, total: 2h 49min 42s
Wall time: 26min 49s


,year,loan_age,avg_credit_score,co_count,dollar_co
0,2003,67.0,735.462844,425.0,47239927.28
1,2015,15.0,754.320344,17.0,3276420.53
2,2007,12.0,721.474947,174.0,32808739.88
3,2003,1.0,729.628530,0.0,0.00
4,2015,16.0,754.371683,20.0,2895264.10
...,...,...,...,...,...
2251,2012,69.0,770.612153,0.0,0.00
2252,2012,66.0,769.964599,1.0,87658.00
2253,2012,70.0,769.715452,0.0,0.00
2254,2012,67.0,770.412095,0.0,0.00


In [16]:
df = _

In [38]:
df.groupby('year').dollar_co.sum()/df.groupby('year').upb_sum.max()

year
1999    0.010831
2000    0.010021
2001    0.008098
2002    0.009312
2003    0.012208
2004    0.025943
2005    0.056303
2006    0.082203
2007    0.089693
2008    0.043820
2009    0.005724
2010    0.001916
2011    0.001169
2012    0.000509
2013    0.000489
2014    0.000458
2015    0.000139
2016    0.000012
dtype: float64

In [ ]:
orig_date
1999    0.010162
2000    0.008756
2001    0.007439
2002    0.007903
2003    0.011534
2004    0.024403
2005    0.054939
2006    0.076471
2007    0.085242
2008    0.042009
2009    0.005343
2010    0.001708
2011    0.001052
2012    0.000468
2013    0.000466
2014    0.000451
2015    0.000124
2016    0.000011